In [ ]:
from controller_modules.create_user_setting_file import CreateUserSetting
from controller_modules.geo_position import GeoPosition

import os
import rasterio
from snappy import ProductIO
import json

from sentinelsat import geojson_to_wkt

import numpy as np
import matplotlib.pyplot as plt

from snapista import Graph
from snapista import operator

from pyproj import Geod
from shapely import wkt

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
#This is an example how to use Snapista to acquire the parameters for a specific operator.
Graph.describe_operators() 
cpRadIndex = Operator('Compactpol-Radar-Vegetation-Index')
cpRadIndex.describe()

In [ ]:
# Here are examples how to use the GeoPosition class to transform coordinate formats

wktPos = GeoPosition().loadWktFromGeojson(os.getcwd() + "/test_files/test.geojson")
geoPoly = GeoPosition().wktToGeojsonShapely(wktPos)
wktPos2 = GeoPosition().geojsonToWktShapely({"coordinates": [[[5.937195, 51.057509], [6.936951, 51.040243], [6.909485, 50.676156], [5.909729, 50.693558], [5.937195, 51.057509]]], "type": "Polygon"})

strCoord = "49.137486,15.163726 49.541000,11.631568 51.162888,12.017920 50.757259,15.674513"
wktSnap = GeoPosition().snapCoordsToWkt(strCoord)

[wktPos,geoPoly,wktPos2,strCoord,wktSnap]

In [ ]:
# This is an example on how to calculate the m² adn km² of a AOI polygon in wkt and WGS84
poly = wkt.loads('POLYGON ((11.383526802062988 48.529563903808594, 11.968979835510254 48.529563903808594, 11.968979835510254 48.324623107910156, 11.383526802062988 48.324623107910156, 11.383526802062988 48.529563903808594, 11.383526802062988 48.529563903808594))')
geod = Geod(ellps="WGS84")
area = abs(geod.geometry_area_perimeter(poly)[0])

print('# Geodesic area: {:12.3f} m²'.format(area))
print('#                {:12.3f} km²'.format(area/1e6))
print(poly)

In [ ]:
# This is an example on how to use snappy to derive a specific band from a sentinel-1 product scene or output of processing operator.
p = ProductIO.readProduct("/codede/Sentinel-1/SAR/SLC/2021/12/15/S1A_IW_SLC__1SDV_20211215T052543_20211215T052610_041015_04DF48_B184.SAFE/manifest.safe")
cpRvi = p.getBand('CpRVI')
w = cpRvi.getRasterWidth()
h = cpRvi.getRasterHeight()
data = np.zeros(w * h, np.float32)
cpRvi.readPixels(0, 0, w, h, data)
p.dispose()
data.shape = h, w
imgplot = plt.imshow(data)
imgplot.write_png(os.getcwd() + '/test_files/test.png')

In [ ]:
# how to add the path to the snap executable to the system path variable
import sys
sys.path.append('/opt/snap/bin/')